In [175]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import altair as alt
import geopandas as gpd
import matplotlib.pyplot as plt
import json
import altair as alt
import dtmapi
import statsmodels.api as sm
from statsmodels.formula.api import ols

### IOM DTM

In [176]:
idp_admin_data = dtmapi.get_idp_admin2_data(CountryName='Sudan',  FromReportingDate='2021-01-01', ToReportingDate='2025-01-01', to_pandas=True)

In [177]:
# read project/data_analysis/sudan/DTM_sdn_SMU - Bi Weekly 13 _ 17122024_V02_Public.xlsx sheet MASTER LIST (ADMIN1)
idp_adm1_2024 = pd.read_excel('sudan/DTM_sdn_SMU - Bi Weekly 13 _ 17122024_V02_Public.xlsx', sheet_name='MASTER LIST (ADMIN1)', skiprows=2)
idp_adm2_2024 = pd.read_excel('sudan/DTM_sdn_SMU - Bi Weekly 13 _ 17122024_V02_Public.xlsx', sheet_name='MASTER LIST (ADMIN2)', skiprows=2)

idp_adm1_2023 = pd.read_excel('sudan/DTM_sdn_Report_Snapshot 13_20231212_V02_Public.xlsx', sheet_name='MASTER LIST (ADMIN1)', skiprows=2)
idp_adm2_2023 = pd.read_excel('sudan/DTM_sdn_Report_Snapshot 13_20231212_V02_Public.xlsx', sheet_name='MASTER LIST (ADMIN2)', skiprows=2)

rename_dict = {
    'STATE OF DISPLACEMET': 'adm1_name',
    'STATE CODE': 'adm1_pcode',
    'LOCALITY OF DISPLACEMENT': 'adm2_name',
    'LOCALITY_CODE': 'adm2_pcode',
    'LOCALITY_ CODE': 'adm2_pcode',
    'HHs': 'affected_idps_hh',
    'IDPs': 'affected_idps_ind'
}

idp_adm1_2023.rename(columns=rename_dict, inplace=True)
idp_adm2_2023.rename(columns=rename_dict, inplace=True)
idp_adm1_2024.rename(columns=rename_dict, inplace=True)
idp_adm2_2024.rename(columns=rename_dict, inplace=True)

In [178]:
idp_origin_adm1_2024 = idp_adm1_2024.iloc[1:]
idp_origin_adm2_2024 = idp_adm2_2024.iloc[1:]
idp_origin_adm1_2023 = idp_adm1_2023.iloc[1:]
idp_origin_adm2_2023 = idp_adm2_2023.iloc[1:]

In [179]:
state_columns = [' Aj Jazirah', ' Central Darfur', ' East Darfur', ' Khartoum',
       ' North Darfur', ' North Kordofan', ' South Darfur', ' South Kordofan',
       ' West Darfur', ' West Kordofan', ' White Nile']

nationality_columns = [' SUDANESE', ' NON SUDANESE']

idp_adm1_2023_state = idp_origin_adm1_2023.melt(
        id_vars=['adm1_name', 'adm1_pcode',  'affected_idps_ind', 'affected_idps_hh'],
        value_vars=state_columns,
        var_name='state_origin',
        value_name='affected_idps_state'
    )

In [180]:
state_columns = [' Aj Jazirah',
       ' Central Darfur', ' East Darfur', ' Khartoum', ' North Darfur',
       ' North Kordofan', ' South Darfur', ' South Kordofan', ' West Darfur',
       ' West Kordofan', ' White Nile']

idp_adm2_2023_state = idp_adm2_2023.melt(
        id_vars=['adm1_name', 'adm1_pcode', 'adm2_name', 'adm2_pcode', 'affected_idps_ind', 'affected_idps_hh'],
        value_vars=state_columns,
        var_name='state_origin',
        value_name='affected_idps_state'
    )


In [181]:
state_columns = ['Aj Jazirah', 'Blue Nile', 'Central Darfur', 'East Darfur', 'Gedaref',
       'Kassala', 'Khartoum', 'North Darfur', 'North Kordofan', 'Northern',
       'Red Sea', 'River Nile', 'Sennar', 'South Darfur', 'South Kordofan',
       'West Darfur', 'West Kordofan', 'White Nile']

idp_adm1_2024_state = idp_origin_adm1_2024.melt(
        id_vars=['adm1_name', 'adm1_pcode',  'affected_idps_ind', 'affected_idps_hh'],
        value_vars=state_columns,
        var_name='state_origin',
        value_name='affected_idps_state'
    )

In [182]:
#filter by adm1_name grand total
idp_origin_adm1_2023 = idp_adm1_2023_state[idp_adm1_2023_state['adm1_name'] == 'Grand Total']
idp_origin_adm1_2024 = idp_adm1_2024_state[idp_adm1_2024_state['adm1_name'] == 'Grand Total']

### IPC 

GeoJson data

In [183]:
ipc = requests.get('https://api.ipcinfo.org/analyses?country=SD&type=A&periods=C&key=3bdbeef9-fc4c-4315-a665-cc56deb11be6').json()

In [184]:
#including obly current data and no projections
for item in ipc:
    response = requests.get(f"https://api.ipcinfo.org/areas/{item['id']}/C?format=geojson&key=3bdbeef9-fc4c-4315-a665-cc56deb11be6")
    if response.status_code == 200:
        try:
            ipc_geoson = response.json()
            with open(f"ipc_geoson_{item['id']}.json", 'w') as f:
                json.dump(ipc_geoson, f)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for item {item['id']}: {e}")
        except Exception as e:
            print(f"Error saving file for item {item['id']}: {e}")
    else:
        print(f"Error fetching data for item {item['id']}: HTTP {response.status_code}")

Error fetching data for item 19191589: HTTP 404
Error fetching data for item 64768802: HTTP 404
Error fetching data for item 74795267: HTTP 404


In [185]:
ipc_2024 = requests.get(f"https://api.ipcinfo.org/areas/68887616/C?format=json&key=3bdbeef9-fc4c-4315-a665-cc56deb11be6").json()

In [186]:
# read project/data_analysis/sudan/ipc_sdn_area_long.csv
ipc_sdn_adm2 = pd.read_csv('sudan/ipc_sdn_area_long.csv')
ipc_sdn_adm2 = ipc_sdn_adm2.iloc[1:]
ipc_sdn_adm1 = pd.read_csv('sudan/ipc_sdn_level1_long.csv')
ipc_sdn_adm1 = ipc_sdn_adm1.iloc[1:]

In [187]:
# filter ipc_sdn_adm2 by Validity period current and year containing 2024 or 2023
ipc_sdn_adm2 = ipc_sdn_adm2[(ipc_sdn_adm2['Validity period'] == 'current') & (ipc_sdn_adm2['Date of analysis'].str.contains('2024')) | 
                            (ipc_sdn_adm2['Validity period'] == 'current') & (ipc_sdn_adm2['Date of analysis'].str.contains('2023'))]
ipc_sdn_adm2 = ipc_sdn_adm2[ipc_sdn_adm2['Phase'] == '3+']


# filter ipc_sdn_adm2 by Validity period current and year containing 2024 or 2023
ipc_sdn_adm1 = ipc_sdn_adm1[(ipc_sdn_adm1['Validity period'] == 'current') & (ipc_sdn_adm1['Date of analysis'].str.contains('2024')) | 
                            (ipc_sdn_adm1['Validity period'] == 'current') & (ipc_sdn_adm1['Date of analysis'].str.contains('2023'))]
ipc_sdn_adm1 = ipc_sdn_adm1[ipc_sdn_adm1['Phase'] == '3+']

In [188]:
ipc_sdn_adm2 = ipc_sdn_adm2[~ipc_sdn_adm2['Area'].str.contains('IDPs')]
ipc_sdn_adm2 = ipc_sdn_adm2[~ipc_sdn_adm2['Area'].str.contains('IDP')]
ipc_sdn_adm2 = ipc_sdn_adm2[~ipc_sdn_adm2['Area'].str.contains('Refugees')]
ipc_sdn_adm2 = ipc_sdn_adm2[~ipc_sdn_adm2['Area'].str.contains('refugees')]


ipc_sdn_adm1 = ipc_sdn_adm1[~ipc_sdn_adm1['Level 1'].str.contains('IDPs')]
ipc_sdn_adm1 = ipc_sdn_adm1[~ipc_sdn_adm1['Level 1'].str.contains('IDP')]
ipc_sdn_adm1 = ipc_sdn_adm1[~ipc_sdn_adm1['Level 1'].str.contains('Refugees')]
ipc_sdn_adm1 = ipc_sdn_adm1[~ipc_sdn_adm1['Level 1'].str.contains('refugees')]

### ACLED

In [262]:
acled = pd.read_csv('sudan/ACLED_2020-01-01-2024-12-31-Sudan.csv')

In [263]:
acled = acled[acled['year'] != 2020]
acled['event_date'] = pd.to_datetime(acled['event_date'])
acled['month_year'] = acled['event_date'].dt.to_period('M')
acled['week'] = acled['event_date'].dt.to_period('W').apply(lambda r: r.start_time)

In [264]:
#Monthly Data
acled_monthly= acled.groupby(['month_year']).agg({'fatalities':'sum', 'event_date':'count'}).reset_index()

#remove riots
#acled_monthly = acled_monthly[acled_monthly['event_type'] != 'Riots']
#acled_monthly['event_type'] = acled_monthly['event_type'].replace('Explosions/Remote violence', 'Explosions')

acled_monthly.to_csv('sudan/acled_monthly.csv', index=False)

acled_monthly['month_year'] = acled_monthly['month_year'].astype(str)

#Weekly Data
acled_weekly = acled.groupby(['week', 'event_type']).agg({'fatalities':'sum', 'event_date':'count'}).reset_index()
acled_weekly['week'] = acled_weekly['week'].astype(str)

In [261]:
# Line plot of total fatalities by month
line = alt.Chart(acled_monthly).mark_line(color='red').encode(
    x='month_year:O',
    y='fatalities:Q',
    tooltip=['month_year', 'fatalities']
).properties(
    title='Total Fatalities and Incidents by Month'
).interactive()

# Bar chart of total incidents by month
bar = alt.Chart(acled_monthly).mark_bar(color='blue').encode(
    x='month_year:O',
    y='event_date:Q',
    tooltip=['month_year', 'event_date']
).interactive()

# Combine both charts into one with a single y-axis
combined_chart = alt.layer(bar, line)

combined_chart


alt.LayerChart(...)

In [239]:
acled_admin1 = acled.groupby(['admin1','year']).agg({'fatalities':'sum', 'event_date':'count'}).reset_index()
acled_admin1.reset_index(drop=True, inplace=True)

acled_admin2 = acled.groupby(['admin2', 'year']).agg({'fatalities':'sum', 'event_date':'count'}).reset_index()
acled_admin2.reset_index(drop=True, inplace=True)

### COMBINED STUFF

### ACLED + IPC

In [ ]:
# Compare admin2 values in acled_admin2 and admin2Name values in idp_admin2_data
acled_admin_values = set(acled_admin1['admin1'])
ipc_admin_values = set(ipc_sdn_adm1['Level 1'])

# Find common and unique values
common_admin = acled_admin_values.intersection(ipc_admin_values)
unique_to_acled = list(acled_admin_values.difference(ipc_admin_values))
unique_to_ipc = list(ipc_admin_values.difference(acled_admin_values))

print("Common admin2 values:", common_admin2)
print("Unique to ACLED admin2 values:", unique_to_acled)
print("Unique to IDP admin2 values:", unique_to_idp)

Common admin2 values: {'Abu Kershola', 'Um Rimta', 'Ar Rahad', 'Al Burgaig', 'Al Idia', 'Al Kamlin', 'Zalingi', 'As Serief', 'Shamal Jabal Marrah', 'Sheikan', 'Buram', 'Bendasi', 'Al Manaqil', 'Tawkar', 'Talawdi', 'Abu Hamad', 'Dilling', 'Janub Al Jazirah', 'Mukjar', 'Saraf Omra', 'Madeinat Kassala', 'Al Lagowa', 'An Nuhud', 'Karrari', 'Al Fao', 'Abu Zabad', 'Rabak', 'Aj Jabalain', 'Al Matama', 'Port Sudan', 'Barbar', 'Tawila', 'Al Qurashi', 'Bara', 'Kateila', 'Kubum', 'Babanusa', 'Wad Bandah', 'Soudari', 'Tulus', 'Al Fasher', 'Kulbus', 'Galabat Ash-Shargiah', 'Gebrat Al Sheikh', 'Dongola', 'Reifi Telkok', 'Kelemando', 'Reifi Wad Elhilaiw', 'Halfa', 'Al Radoum', 'Beliel', 'Al Buhaira', 'Basundah', 'Nyala Shimal', 'Um Dam Haj Ahmed', 'Ad Damar', 'Halfa Aj Jadeedah', 'Khartoum', 'Al Meiram', 'Kernoi', 'Bahr Al Arab', 'Sirba', 'As Sunut', 'Adila', 'Ad Dabbah', 'Dordieb', 'Al Malha', 'Wad Al Mahi', 'Jebel Moon', 'Reifi Kassla', 'Al Khiwai', 'Al Lait', 'Um Bada', 'Azum', 'Nitega', 'Abu Kari

In [ ]:
acled_adm1 = acled_admin1.groupby('admin1').agg({'fatalities': 'sum', 'event_date': 'sum'}).reset_index()
acled_adm1.head(50)

#rename Al Jazirah to Aj Jazirah
acled_adm1['admin1'] = acled_adm1['admin1'].replace('Al Jazirah', 'Aj Jazirah')

In [ ]:
ipc_sdn_adm1_2024 = ipc_sdn_adm1[ipc_sdn_adm1['Date of analysis'].str.contains('Apr 2024')]
ipc_sdn_adm1_2024

,Date of analysis,Country,Total country population,Level 1,Validity period,From,To,Phase,Number,Percentage
260,Apr 2024,SDN,47208125,South Kordofan,current,2024-04-01,2024-05-31,3+,1018272,0.48
281,Apr 2024,SDN,47208125,East Darfur,current,2024-04-01,2024-05-31,3+,686566,0.39
302,Apr 2024,SDN,47208125,Blue Nile,current,2024-04-01,2024-05-31,3+,734784,0.5
323,Apr 2024,SDN,47208125,Central Darfur,current,2024-04-01,2024-05-31,3+,1167817,0.58
344,Apr 2024,SDN,47208125,Gedaref,current,2024-04-01,2024-05-31,3+,966142,0.31
365,Apr 2024,SDN,47208125,Kassala,current,2024-04-01,2024-05-31,3+,1130199,0.38
386,Apr 2024,SDN,47208125,North Kordofan,current,2024-04-01,2024-05-31,3+,981111,0.43
407,Apr 2024,SDN,47208125,West Darfur,current,2024-04-01,2024-05-31,3+,765187,0.59
428,Apr 2024,SDN,47208125,Red Sea,current,2024-04-01,2024-05-31,3+,538962,0.3
449,Apr 2024,SDN,47208125,Sennar,current,2024-04-01,2024-05-31,3+,1311693,0.49


In [ ]:
acled_ipc_adm1 = pd.merge(acled_adm1, ipc_sdn_adm1_2024, left_on=['admin1'], right_on=['Level 1'], how='inner')
acled_ipc_adm1

,admin1,fatalities,event_date,Date of analysis,Country,Total country population,Level 1,Validity period,From,To,Phase,Number,Percentage
0,Aj Jazirah,3692,1538,Apr 2024,SDN,47208125,Aj Jazirah,current,2024-04-01,2024-05-31,3+,2853931,0.53
1,Blue Nile,249,98,Apr 2024,SDN,47208125,Blue Nile,current,2024-04-01,2024-05-31,3+,734784,0.5
2,Central Darfur,495,323,Apr 2024,SDN,47208125,Central Darfur,current,2024-04-01,2024-05-31,3+,1167817,0.58
3,East Darfur,139,132,Apr 2024,SDN,47208125,East Darfur,current,2024-04-01,2024-05-31,3+,686566,0.39
4,Gedaref,73,164,Apr 2024,SDN,47208125,Gedaref,current,2024-04-01,2024-05-31,3+,966142,0.31
5,Kassala,3,89,Apr 2024,SDN,47208125,Kassala,current,2024-04-01,2024-05-31,3+,1130199,0.38
6,Khartoum,8768,5783,Apr 2024,SDN,47208125,Khartoum,current,2024-04-01,2024-05-31,3+,2541541,0.49
7,North Darfur,4845,1341,Apr 2024,SDN,47208125,North Darfur,current,2024-04-01,2024-05-31,3+,1438277,0.59
8,North Kordofan,1460,459,Apr 2024,SDN,47208125,North Kordofan,current,2024-04-01,2024-05-31,3+,981111,0.43
9,Northern,46,115,Apr 2024,SDN,47208125,Northern,current,2024-04-01,2024-05-31,3+,416243,0.29


In [ ]:
#correlation between fatalities and Percentage
acled_ipc_adm1['fatalities'].corr(acled_ipc_adm1['Number'])

0.6461436976967655

In [ ]:
# Create the chart with the selection and regression line
chart = alt.Chart(acled_ipc_adm1).mark_circle().encode(
    x=alt.X('fatalities:Q', scale=alt.Scale(type='log')),
    y=alt.Y('Number:Q', scale=alt.Scale(type='log')),
    color='admin1:N',
    size=alt.Size('fatalities:Q', scale=alt.Scale(range=[10, 100])),
    tooltip=['admin1', 'fatalities', 'Number']
).properties(
    title='Fatalities vs IPC by Admin1 in Sudan'
).interactive()

chart

alt.Chart(...)

#### ACLED + IOM

In [194]:
# Compare admin2 values in acled_admin2 and admin2Name values in idp_admin2_data
acled_admin2_values = set(acled_admin2['admin2'])
idp_admin2_values = set(idp_admin_data['admin2Name'])

# Find common and unique values
common_admin2 = acled_admin2_values.intersection(idp_admin2_values)
unique_to_acled = acled_admin2_values.difference(idp_admin2_values)
unique_to_idp = idp_admin2_values.difference(acled_admin2_values)

print("Common admin2 values:", common_admin2)
print("Unique to ACLED admin2 values:", unique_to_acled)
print("Unique to IDP admin2 values:", unique_to_idp)

Common admin2 values: {'Abu Kershola', 'Um Rimta', 'Ar Rahad', 'Al Burgaig', 'Al Idia', 'Al Kamlin', 'Zalingi', 'As Serief', 'Shamal Jabal Marrah', 'Sheikan', 'Buram', 'Bendasi', 'Al Manaqil', 'Tawkar', 'Talawdi', 'Abu Hamad', 'Dilling', 'Janub Al Jazirah', 'Mukjar', 'Saraf Omra', 'Madeinat Kassala', 'Al Lagowa', 'An Nuhud', 'Karrari', 'Al Fao', 'Abu Zabad', 'Rabak', 'Aj Jabalain', 'Al Matama', 'Port Sudan', 'Barbar', 'Tawila', 'Al Qurashi', 'Bara', 'Kateila', 'Kubum', 'Babanusa', 'Wad Bandah', 'Soudari', 'Tulus', 'Al Fasher', 'Kulbus', 'Galabat Ash-Shargiah', 'Gebrat Al Sheikh', 'Dongola', 'Reifi Telkok', 'Kelemando', 'Reifi Wad Elhilaiw', 'Halfa', 'Al Radoum', 'Beliel', 'Al Buhaira', 'Basundah', 'Nyala Shimal', 'Um Dam Haj Ahmed', 'Ad Damar', 'Halfa Aj Jadeedah', 'Khartoum', 'Al Meiram', 'Kernoi', 'Bahr Al Arab', 'Sirba', 'As Sunut', 'Adila', 'Ad Dabbah', 'Dordieb', 'Al Malha', 'Wad Al Mahi', 'Jebel Moon', 'Reifi Kassla', 'Al Khiwai', 'Al Lait', 'Um Bada', 'Azum', 'Nitega', 'Abu Kari

In [195]:
idp_admin_data['admin2Name'] = idp_admin_data['admin2Name'].replace({
    "Ad Du'ayn": 'Ad Duayn',
    'Um Durman': 'Omdurman',
    'At Tawisha': 'El Taweisha',
    "Gala'a Al Nahal": 'Galaa Al Nahal',
    "Jubayt Elma'aadin": 'Jubayt Elmaaadin',
    'As Sunta': 'Sunta',
    'Ar Rashad': 'Rashad',
    'Ad Dinder': 'Dinder',
    'As Salam / Ar Rawat': 'As Salam/Ar Rawat',
    'Al Galabat Al Gharbyah - Kassab': 'Al Galabat Al Gharbyah-Kassab',
    'Ar Rusayris': 'El Roseires',
    'Ag Geneina': 'El Geneina',
    'Baw': 'Bau',
    'Merwoe': 'Merowe',
    'As Salam - WK': 'El Salam'
})

idp_admin_data['admin1Name'] = idp_admin_data['admin1Name'].replace('Al Jazirah', 'Aj Jazirah')

In [196]:
idp_admin2_2024 = idp_admin_data[(idp_admin_data['yearReportingDate'] == 2024) & (idp_admin_data['monthReportingDate'] == 12) & (idp_admin_data['operation'] == 'Armed Clashes in Sudan (Overview)') & (idp_admin_data['roundNumber'] == 18)]
idp_admin2_2023 = idp_admin_data[(idp_admin_data['yearReportingDate'] == 2023) & (idp_admin_data['monthReportingDate'] == 12) & (idp_admin_data['operation'] == 'Armed Clashes in Sudan (Overview)')]
idp_admin2_2022 = idp_admin_data[(idp_admin_data['yearReportingDate'] == 2022) & (idp_admin_data['monthReportingDate'] == 3) & (idp_admin_data['operation'] == 'Darfur conflict')]
idp_admin2_2021 = idp_admin_data[(idp_admin_data['yearReportingDate'] == 2021) & (idp_admin_data['monthReportingDate'] == 12) & (idp_admin_data['operation'] == 'Darfur conflict')]

idp_admin2 = pd.concat([idp_admin2_2024, idp_admin2_2023, idp_admin2_2022, idp_admin2_2021])

In [197]:
idp_admin2_yearly = idp_admin2.groupby(['yearReportingDate']).agg({'numPresentIdpInd':'sum'}).reset_index()
idp_admin2_yearly

,yearReportingDate,numPresentIdpInd
0,2021,3086553
1,2022,3779487
2,2023,9052822
3,2024,11532774


In [198]:
# Merged datasets on adm level 1 and 2
idp_acled_adm2 = pd.merge(idp_admin2, acled_admin2, left_on=['admin2Name', 'yearReportingDate'], right_on=['admin2', 'year'], how='inner')
idp_acled_adm1= idp_acled_adm2.groupby(['admin1Name', 'year']).agg({'numPresentIdpInd':'sum', 'fatalities':'sum', 'event_date':'sum'}).reset_index()

#save both at xlsx
idp_acled_adm1.to_excel('sudan/idp_acled_adm1.xlsx', index=False)
idp_acled_adm2.to_excel('sudan/idp_acled_adm2.xlsx', index=False)

In [199]:
idp_acled_adm1_2023 = idp_acled_adm1[idp_acled_adm1['year'] == 2023]
idp_acled_adm1_2024 = idp_acled_adm1[idp_acled_adm1['year'] == 2024]

Look at state of origin

In [200]:
idp_origin_adm1_2023 = idp_origin_adm1_2023.drop(columns=['adm1_name', 'adm1_pcode', 'affected_idps_ind', 'affected_idps_hh'])
idp_origin_adm1_2024 = idp_origin_adm1_2024.drop(columns=['adm1_name', 'adm1_pcode', 'affected_idps_ind', 'affected_idps_hh'])

idp_origin_adm1_2023['state_origin'] = idp_origin_adm1_2023['state_origin'].str.strip()

In [201]:
# merge idp_adm1_2023 with idp_acled_adm1_2023 on adm1_name and state_origin
acled_idp_2023 = pd.merge(idp_origin_adm1_2023, idp_acled_adm1_2023, left_on=['state_origin'], right_on=['admin1Name'], how='inner')
acled_idp_2024 = pd.merge(idp_origin_adm1_2024, idp_acled_adm1_2024, left_on=['state_origin'], right_on=['admin1Name'], how='inner')

#add a year column and row bind both
acled_idp_2023['year'] = 2023
acled_idp_2024['year'] = 2024

acled_idp_origin = pd.concat([acled_idp_2023, acled_idp_2024])


In [202]:
alt.Chart(acled_idp_origin).mark_circle().encode(
    x=alt.X('affected_idps_state:Q', scale=alt.Scale(type='log')),
    y=alt.Y('fatalities:Q', scale=alt.Scale(type='log')),
    color='year:O',
    size='fatalities:Q'
).properties(
    title='Total Fatalities by Month'
).interactive()


alt.Chart(...)

In [203]:
idp_acled_adm1['numPresentIdpInd'].corr(idp_acled_adm1['event_date'])
correlations_by_year = idp_acled_adm1.groupby('year').apply(lambda x: x['numPresentIdpInd'].corr(x['event_date']))
correlations_by_year

year
2023   -0.201296
2024   -0.118161
dtype: float64

In [204]:
idp_acled_adm2['numPresentIdpInd'].corr(idp_acled_adm2['event_date'])
correlations_by_year = idp_acled_adm2.groupby('year').apply(lambda x: x['numPresentIdpInd'].corr(x['event_date']))
correlations_by_year

year
2023   -0.003608
2024    0.152145
dtype: float64

In [205]:
model = ols('numPresentIdpInd ~ event_date', data=idp_acled_adm2).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:       numPresentIdpInd   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.583
Date:                Mon, 06 Jan 2025   Prob (F-statistic):              0.209
Time:                        18:05:08   Log-Likelihood:                -4002.3
No. Observations:                 315   AIC:                             8009.
Df Residuals:                     313   BIC:                             8016.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   5.845e+04   4763.895     12.269      0.0

In [206]:
#remove admin2Name reifi gharb kassala  and Jubayt Elmaaadin
#idp_acled = idp_acled[idp_acled['admin2Name'] != 'Reifi Gharb Kassala']
#idp_acled = idp_acled[idp_acled['admin2Name'] != 'Jubayt Elmaaadin']

In [207]:
idp_acled_graph = idp_acled_adm2[(idp_acled_adm2['fatalities'] > 0)]

In [208]:
# Create a selection for the year
year_dropdown = alt.binding_select(options=idp_acled_graph['year'].unique(), name='Year ')
year_select = alt.selection_single(fields=['year'], bind=year_dropdown, value=idp_acled_graph['year'].unique()[0])

# Create the chart with the selection and regression line
chart = alt.Chart(idp_acled_graph).mark_circle().encode(
    x=alt.X('fatalities:Q', scale=alt.Scale(type='log')),
    y=alt.Y('numPresentIdpInd:Q', scale=alt.Scale(type='log')),
    color='admin1Name:N',
    tooltip=['admin1Name', 'fatalities', 'numPresentIdpInd']
).properties(
    title='Nr. of incidents vs Number of IDPs by Admin1 in Sudan'
).add_selection(
    year_select
).transform_filter(
    year_select
).interactive()

# Add regression line
regression_line = chart.transform_regression(
    'fatalities', 'numPresentIdpInd', groupby=['year']
).mark_line(color='red')

# Combine chart and regression line
final_chart = chart + regression_line

final_chart


/var/folders/w9/4s00gf1j7sl8k190_53qgb7c0000gn/T/ipykernel_53003/1691761154.py:3: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use selection_point instead.
  year_select = alt.selection_single(fields=['year'], bind=year_dropdown, value=idp_acled_graph['year'].unique()[0])
/var/folders/w9/4s00gf1j7sl8k190_53qgb7c0000gn/T/ipykernel_53003/1691761154.py:13: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.LayerChart(...)